# W207 Final Project
Erika, Jen Jen, Geoff, Leslie

(In Python 3)

As of 3/35

Outline:

* Data Pre-Processing  
* Simple Feature Selection
* Basline Models
* Possible Approaches
   

# Section 1 Loading and Processing Data

In [1]:
## Import Libraries ##
import json
from pprint import pprint
from pandas import *
from pandas.io.json import json_normalize


# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import recall_score

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

In [69]:
## Get Data ##

#reference on data: https://www.kaggle.com/c/random-acts-of-pizza/data
# pull in the training and test data
with open('/Users/erikaananda/Documents/MIDS/W207/Final Project/data/train.json', encoding='utf-8') as data_file:
#with open('/home/levi/Documents/W207_Proj/data/train.json', encoding='utf-8') as data_file:
    trainData = json.loads(data_file.read())   
with open('/Users/erikaananda/Documents/MIDS/W207/Final Project/data/test.json', encoding='utf-8') as data_file:
#with open('/home/levi/Documents/W207_Proj/data/train.json', encoding='utf-8') as data_file:
    testData = json.loads(data_file.read())    

# create a dev data set 
devData = trainData[0:1000]
trainData = trainData[1000:]

# baseline info
print("Percent of people who got pizza:", round(sum(trainLabel)/len(trainLabel),3))

# show how the data looks in its original format
#pprint("data in json format:")
#pprint(trainData[1])

# create a normalized view
allTData = json_normalize(trainData)
print("\nSize of the normalized Data:", allTData.shape)
print("\nnormalized data columns:", list(allTData))

allDData = json_normalize(devData)


Percent of people who got pizza: 0.241

Size of the normalized Data: (3040, 32)

normalized data columns: ['giver_username_if_known', 'number_of_downvotes_of_request_at_retrieval', 'number_of_upvotes_of_request_at_retrieval', 'post_was_edited', 'request_id', 'request_number_of_comments_at_retrieval', 'request_text', 'request_text_edit_aware', 'request_title', 'requester_account_age_in_days_at_request', 'requester_account_age_in_days_at_retrieval', 'requester_days_since_first_post_on_raop_at_request', 'requester_days_since_first_post_on_raop_at_retrieval', 'requester_number_of_comments_at_request', 'requester_number_of_comments_at_retrieval', 'requester_number_of_comments_in_raop_at_request', 'requester_number_of_comments_in_raop_at_retrieval', 'requester_number_of_posts_at_request', 'requester_number_of_posts_at_retrieval', 'requester_number_of_posts_on_raop_at_request', 'requester_number_of_posts_on_raop_at_retrieval', 'requester_number_of_subreddits_at_request', 'requester_received_p

In [89]:
## Create subsets of data for analysis ###

# create a flat dataset without the subreddits list
flatData = allTData.drop('requester_subreddits_at_request', 1)
# create a separate dataset with just subreddits, indexed on request id
# we can creata a count vector on the words, run Naive Bayes against it, 
# and add the probabilities to our flat dataset
subredTData = allTData[['request_id','requester_subreddits_at_request']]
subredTData.set_index('request_id', inplace=True)

subredDData= allDData[['request_id','requester_subreddits_at_request']]
subredDData.set_index('request_id', inplace=True)

# our training labels
trainLabel = allTData['requester_received_pizza']

devLabel = allDData['requester_received_pizza']

# what do these look like?
#print(list(flatData))
print(subredTData.shape)
#print(subredTData['requester_subreddits_at_request'][1])

# create a corpus of subreddits to vectorize
trainCorpus = []
rTCorpus = []
rDCorpus = []

for index in range(len(subredTData)):
    trainCorpus.append(' '.join(subredTData['requester_subreddits_at_request'][index]))
    rTCorpus.append(' '.join(subredTData['requester_subreddits_at_request'][index]))

devCorpus = []
for index in range(len(subredDData)):
    devCorpus.append(' '.join(subredDData['requester_subreddits_at_request'][index]))
    rDCorpus.append(' '.join(subredDData['requester_subreddits_at_request'][index]))




(3040, 1)


In [4]:
# combine all text sources into a single corpus
fldTText = allTData[['request_id','request_text', 'request_text_edit_aware', 'request_title']]
fldDText = allDData[['request_id','request_text', 'request_text_edit_aware', 'request_title']]

trainCorpus = []
for index in range(len(subredTData)):
    a = ' '.join(subredTData['requester_subreddits_at_request'][index])
    b = (a, fldTText['request_text'][index], fldTText['request_text_edit_aware'][index],
        fldTText['request_title'][index])
    trainCorpus.append(' '.join(b))

devCorpus = []
for index in range(len(subredDData)):
    a = ' '.join(subredDData['requester_subreddits_at_request'][index])
    b = (a, fldDText['request_text'][index], fldDText['request_text_edit_aware'][index],
         fldDText['request_title'][index])
    devCorpus.append(' '.join(b))

# Print 3 examples  
print (trainCorpus[:3])
labels = trainLabel.astype(int)
labels = list(labels)
print(labels[:3])
print('-'*75)

print ('\n' , devCorpus[:3])
labels_dev = devLabel.astype(int)
labels_dev = list(labels_dev)
print(labels_dev[:3])

["FoodstuffsAllAround IAmA RandomActsOfCookies RandomActsofCards RandomKindness Random_Acts_Of_Pizza comiccon cosplay cosplayers So it's been a while since it's happened, but yeah. Just got dumped by a girl I like... My brother has used up pretty much everything in the freezer. I would really appreciate a pizza right now... So it's been a while since it's happened, but yeah. Just got dumped by a girl I like... My brother has used up pretty much everything in the freezer. I would really appreciate a pizza right now... [Request] Just got dumped, no food in the freezer. Pizza?", "AskReddit Guitar Jazz Music NSFW_GIF Psychonaut RoomPorn StAugustine TwoXChromosomes WTF YouShouldKnow atheism aww bakedart catpictures cats crochet dubstep ents entwives food funny gonewild hiphopheads listentothis meetup offbeat pics realpics self sex tattoos treecomics treemusic trees videos vinyl zombies My boyfriend and I live in Saint Augustine, Florida and have been having a rough time financially the past

# Section 2. Simple Feature Selection and Pre-Processing

In [34]:
# Simple Pre-Processing

def data_preprocessor(s):
    """
    Note: this function pre-processors data:
    (1) removes non-alpha characters
    (2) converts digits to 'number'
    (3) regularizes spaces (although CountVectorizer ignores this unless they are part of words)
    (4) reduces word size to n
    """

    s = [re.sub(r'[?|$|.|!|@|\n|(|)|<|>|_|-|,|\']',r' ',s) for s in s] # strip out non-alpha numeric char, replace with space
    s = [re.sub(r'\d+',r'number ',s) for s in s] # convert digits to number
    s = [re.sub(r' +',r' ',s) for s in s] # convert multiple spaces to single space
    
    # This sets word size to n=5
    num = 5
    def set_word(s):
        temp = []
        for s in s:
            x = s.split()
            z = [elem[:num] for elem in x]
            z = ' '.join(z)
            temp.append(z)       
        return temp
    
    s = set_word(s)
    
    return s



In [42]:
stop = {'I','about', 'a', 'an', 'are', 'as', 'at', 'be', 'by', 'com', 'for', 'from', 'how',
        'in','is','it','of','on','or','that','the','this','to','was','what','when','where',
        'who','will', 'with', 'the','www'}
stop_long = {"a","able","about","above","abst","accordance","according","accordingly","across",
        "act","actually","added","adj","affected","affecting","affects","after","afterwards",
        "again","against","ah","all","almost","alone","along","already","also","although",
        "always","am","among","amongst","an","and","announce","another","any","anybody",
        "anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently",
        "approximately","are","aren","arent","arise","around","as","aside","ask","asking",
        "at","auth","available","away","awfully","b","back","be","became","because","become",
        "becomes","becoming","been","before","beforehand","begin","beginning","beginnings",
        "begins","behind","being","believe","below","beside","besides","between","beyond",
        "biol","both","brief","briefly","but","by","c","ca","came","can","cannot","can't",
        "cause","causes","certain","certainly","co","com","come","comes","contain","containing",
        "contains","could","couldnt","d","date","did","didn't","different","do","does",
        "doesn't","doing","done","don't","down","downwards","due","during","e","each","ed",
        "edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending",
        "enough","especially","et","et-al","etc","even","ever","every","everybody","everyone",
        "everything","everywhere","ex","except","f","far","few","ff","fifth","first","five",
        "fix","followed","following","follows","for","former","formerly","forth","found","four",
        "from","further","furthermore","g","gave","get","gets","getting","give","given","gives",
        "giving","go","goes","gone","got","gotten","h","had","happens","hardly","has","hasn't",
        "have","haven't","having","he","hed","hence","her","here","hereafter","hereby","herein",
        "heres","hereupon","hers","herself","hes","hi","hid","him","himself","his","hither",
        "home","how","howbeit","however","hundred","i","id","ie","if","i'll","im","immediate",
        "immediately","importance","important","in","inc","indeed","index","information",
        "instead","into","invention","inward","is","isn't","it","itd","it'll","its","itself",
        "i've","j","just","k","keep","keeps","kept","kg","km","know","known","knows","l",
        "largely","last","lately","later","latter","latterly","least","less","lest","let",
        "lets","like","liked","likely","line","little","ll","look","looking","looks","ltd","m",
        "made","mainly","make","makes","many","may","maybe","me","mean","means","meantime",
        "meanwhile","merely","mg","might","million","miss","ml","more","moreover","most",
        "mostly","mr","mrs","much","mug","must","my","myself","n","na","name","namely","nay",
        "nd","near","nearly","necessarily","necessary","need","needs","neither","never",
        "nevertheless","new","next","nine","ninety","no","nobody","non","none","nonetheless",
        "noone","nor","normally","nos","not","noted","nothing","now","nowhere","o","obtain",
        "obtained","obviously","of","off","often","oh","ok","okay","old","omitted","on","once",
        "one","ones","only","onto","or","ord","other","others","otherwise","ought","our",
        "ours","ourselves","out","outside","over","overall","owing","own","p","page","pages",
        "part","particular","particularly","past","per","perhaps","placed","please","plus",
        "poorly","possible","possibly","potentially","pp","predominantly","present",
        "previously","primarily","probably","promptly","proud","provides","put","q","que",
        "quickly","quite","qv","r","ran","rather","rd","re","readily","really","recent",
        "recently","ref","refs","regarding","regardless","regards","related","relatively",
        "research","respectively","resulted","resulting","results","right","run","s","said",
        "same","saw","say","saying","says","sec","section","see","seeing","seem","seemed",
        "seeming","seems","seen","self","selves","sent","seven","several","shall","she","shed",
        "she'll","shes","should","shouldn't","show","showed","shown","showns","shows",
        "significant","significantly","similar","similarly","since","six","slightly","so",
        "some","somebody","somehow","someone","somethan","something","sometime","sometimes",
        "somewhat","somewhere","soon","sorry","specifically","specified","specify",
        "specifying","still","stop","strongly","sub","substantially","successfully","such",
        "sufficiently","suggest","sup","sure","t","take","taken","taking","tell","tends","th",
        "than","thank","thanks","thanx","that","that'll","thats","that've","the","their",
        "theirs","them","themselves","then","thence","there","thereafter","thereby","thered",
        "therefore","therein","there'll","thereof","therere","theres","thereto","thereupon",
        "there've","these","they","theyd","they'll","theyre","they've","think","this","those",
        "thou","though","thoughh","thousand","throug","through","throughout","thru","thus",
        "til","tip","to","together","too","took","toward","towards","tried","tries","truly",
        "try","trying","ts","twice","two","u","un","under","unfortunately","unless","unlike",
        "unlikely","until","unto","up","upon","ups","us","use","used","useful","usefully",
        "usefulness","uses","using","usually","v","value","various","ve","very","via","viz",
        "vol","vols","vs","w","want","wants","was","wasnt","way","we","wed","welcome","we'll",
        "went","were","werent","we've","what","whatever","what'll","whats","when","whence",
        "whenever","where","whereafter","whereas","whereby","wherein","wheres","whereupon",
        "wherever","whether","which","while","whim","whither","who","whod","whoever","whole",
        "who'll","whom","whomever","whos","whose","why","widely","willing","wish","with","within",
        "without","wont","words","world","would","wouldnt","www","x","y","yes","yet","you",
        "youd","you'll","your","youre","yours","yourself","yourselves","you've","z","zero"}

In [6]:
# Set up the data with CountVectorizer

#vectorizer = CountVectorizer(lowercase=True, strip_accents='unicode',stop_words='english')

vectorizer = CountVectorizer(min_df=1,lowercase=True)
tVector = vectorizer.fit_transform(trainCorpus)
dVector = vectorizer.transform(devCorpus)

print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", tVector.shape[1])
print ("First 5 feature Names:", vectorizer.get_feature_names()[1:6], "\n")

tVector_p = vectorizer.fit_transform(data_preprocessor(trainCorpus))
dVector_p = vectorizer.transform(data_preprocessor(devCorpus))

print ('\nPre-Processed data:')
print ("The size of the vocabulary for the training text data is", tVector_p.shape[1])
print ("First 5 feature Names:", vectorizer.get_feature_names()[1:6], "\n")





Raw data:
The size of the vocabulary for the training text data is 17213
First 5 feature Names: ['000', '0000', '0011011001111000', '00243364', '00pm'] 


Pre-Processed data:
The size of the vocabulary for the training text data is 10491
First 5 feature Names: ['aaaaa', 'aan', 'ab', 'aback', 'aband'] 



In [14]:
# try it with bigrams 
biVectorizer = CountVectorizer(analyzer = 'word', ngram_range=(2, 2), 
                               token_pattern=r'\b\w+\b', min_df=1)
biTrainVector = biVectorizer.fit_transform(trainCorpus)
biDVector = biVectorizer.transform(data_preprocessor(devCorpus))

print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", biTrainVector.shape[1])
print ("First 5 feature Names:", biVectorizer.get_feature_names()[1:6], "\n")

biVectorizer_p = CountVectorizer(analyzer = 'word', ngram_range=(2, 2), 
                                 token_pattern=r'\b\w+\b', min_df=1)
biTrainVector_p = biVectorizer_p.fit_transform(data_preprocessor(trainCorpus))
biDVector_p = biVectorizer_p.transform(data_preprocessor(devCorpus))
print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", biTrainVector_p.shape[1])
print ("First 5 feature Names:", biVectorizer_p.get_feature_names()[1:6], "\n")




Raw data:
The size of the vocabulary for the training text data is 122674
First 5 feature Names: ['0 00', '0 00243364', '0 012468', '0 02', '0 024856'] 


Raw data:
The size of the vocabulary for the training text data is 113396
First 5 feature Names: ['a activ', 'a airso', 'a am', 'a amp', 'a anyth'] 



In [53]:
# try it with trigrams 

triVectorizer = CountVectorizer(analyzer = 'word', ngram_range=(3, 3), 
                                token_pattern=r'\b\w+\b', min_df=1,stop_words = stop)
triTrainVector = triVectorizer.fit_transform(trainCorpus)
triDVector = triVectorizer.transform(data_preprocessor(devCorpus))

print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", triTrainVector.shape[1])
print ("First 5 feature Names:", triVectorizer.get_feature_names()[1:6], "\n")

triVectorizer_p = CountVectorizer(analyzer = 'word', ngram_range=(2, 2), 
                                  token_pattern=r'\b\w+\b', min_df=1,stop_words = stop)
triTrainVector_p = triVectorizer_p.fit_transform(data_preprocessor(trainCorpus))
triDVector_p = triVectorizer_p.transform(data_preprocessor(devCorpus))
print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", triTrainVector_p.shape[1])
print ("First 5 feature Names:", triVectorizer_p.get_feature_names()[1:6], "\n")



Raw data:
The size of the vocabulary for the training text data is 218794
First 5 feature Names: ['0 00 even', '0 00 left', '0 00243364 bitcoins', '0 012468 0', '0 02 bank'] 


Raw data:
The size of the vocabulary for the training text data is 116681
First 5 feature Names: ['aa meeti', 'aaaaa actua', 'aaaaa adven', 'aaaaa agnos', 'aaaaa ainbo'] 



In [90]:
#tf-idf vectorizer
stop = {'I','about', 'a', 'aa', 'aaa','aaaa','aaaaa','ab','aan','an', 'are', 'as', 'at', 'be', 'by', 'com', 'for', 'from', 'how',
        'in','is','it','of','on','or','that','the','this','to','was','what','when','where',
        'who','will', 'with', 'the','www'}
vectorizer = TfidfVectorizer(min_df=1,ngram_range=(2, 6),stop_words = stop)
trainVector = vectorizer.fit_transform(data_preprocessor(trainCorpus))
numF = trainVector.shape[1]
print("Number of features =", numF)
devVector = vectorizer.transform(data_preprocessor(devCorpus))

    
print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", devVector.shape[1])
print ("First 5 feature Names:", vectorizer.get_feature_names()[1:30], "\n")

#subreddits
Rvectorizer = TfidfVectorizer(min_df=1, stop_words = stop)
RtrainVector = Rvectorizer.fit_transform(data_preprocessor(rTCorpus))
numF = RtrainVector.shape[1]
print("Number of features =", numF)
RdevVector = Rvectorizer.transform(data_preprocessor(rDCorpus))
    
print ('\nRaw data:')
print ("The size of the vocabulary for the training text data is", numF)
print ("First 5 feature Names:", Rvectorizer.get_feature_names()[1:30], "\n")


Number of features = 206106

Raw data:
The size of the vocabulary for the training text data is 206106
First 5 feature Names: ['aband achie alien', 'aband achie alien askre', 'aband achie alien askre bizar', 'aband achie alien askre bizar bugwa', 'aband advan', 'aband advan advic', 'aband advan advic askre', 'aband advan advic askre diabl', 'aband advan advic askre diabl guild', 'aband advic', 'aband advic advic', 'aband advic advic arche', 'aband advic advic arche askan', 'aband advic advic arche askan askco', 'aband advic advic askre', 'aband advic advic askre bgsu', 'aband advic advic askre bgsu bunde', 'aband advic album', 'aband advic album andro', 'aband advic album andro andro', 'aband advic album andro andro askre', 'aband advic album antij', 'aband advic album antij anyth', 'aband advic album antij anyth askre', 'aband advic andro', 'aband advic andro askre', 'aband advic andro askre autod', 'aband advic andro askre autod autos', 'aband advic anima'] 

Number of features = 447

# Section 3. Baseline Models

## Logistic Regression

In [16]:
# Logistic Regression

C = 0.01 #(For now)

modelLogit = LogisticRegression(penalty='l2', C=C)
modelLogit.fit(tVector,trainLabel)
logitScore = round(modelLogit.score(dVector, devLabel), 4)
print("For C = ", C, "Logistic regression accuracy:", logitScore)

modelLogit.fit(tVector_p,trainLabel)
logitScore = round(modelLogit.score(dVector_p, devLabel), 4)
print("For C = ", C, "Logistic regression (processed data) accuracy:", logitScore)


For C =  0.01 Logistic regression accuracy: 0.736
For C =  0.01 Logistic regression (processed data) accuracy: 0.73


In [101]:
# Logistic Regression: bigrams
# fit a logistic regression model to our bigrams, using l2 regularization and C = .5
biModelLogit = LogisticRegression(penalty='l2', C=.5)
biModelLogit.fit(biTrainVector, trainLabel)
    # print out the features with the biggest weights for each category,
    # across all categories    
logitScore = round(biModelLogit.score(biDVector, devLabel), 4)
print("For C = ", C, "Logistic regression accuracy:", logitScore)

biModelLogit_p = LogisticRegression(penalty='l2', C=.5)
biModelLogit_p.fit(biTrainVector_p, trainLabel)

# print out the classification report
print("Classification report:\n")
devPred = biModelLogit_p.predict(biDVector_p)
print(classification_report(devLabel, devPred))

# print the accuracy score
logitScore = round(biModelLogit_p.score(biDVector_p, devLabel), 4)
print("For C = ", C, "Logistic regression accuracy:", logitScore)

For C =  0.01 Logistic regression accuracy: 0.742
Classification report:

             precision    recall  f1-score   support

      False       0.76      0.93      0.83       740
       True       0.41      0.15      0.22       260

avg / total       0.67      0.72      0.67      1000

For C =  0.01 Logistic regression accuracy: 0.724


In [102]:
# Logistic Regression: trigrams
# fit a logistic regression model to our bigrams, using l2 regularization and C = .5
triModelLogit = LogisticRegression(penalty='l2', C=.5)
triModelLogit.fit(triTrainVector, trainLabel)
    # print out the features with the biggest weights for each category,
    # across all categories    
logitScore = round(triModelLogit.score(triDVector, devLabel), 4)
print("For C = ", C, "Logistic regression accuracy:", logitScore)

triModelLogit_p = LogisticRegression(penalty='l2', C=.5)
triModelLogit_p.fit(triTrainVector_p, trainLabel)

# print out the classification report
print("Classification report:\n")
devPred = triModelLogit_p.predict(triDVector_p)
print(classification_report(devLabel, devPred))
    
# print the accuracy score
logitScore = round(triModelLogit_p.score(triDVector_p, devLabel), 4)
print("For C = ", C, "Logistic regression accuracy:", logitScore)

For C =  0.01 Logistic regression accuracy: 0.738
Classification report:

             precision    recall  f1-score   support

      False       0.76      0.96      0.85       740
       True       0.54      0.12      0.19       260

avg / total       0.70      0.74      0.68      1000

For C =  0.01 Logistic regression accuracy: 0.744


In [97]:
#tf-idf # how does it look?
reg = [100]
for c in reg: 
    modelLogit = LogisticRegression(penalty='l2', C=c )
    modelLogit.fit(trainVector, trainLabel)
    devPred = modelLogit.predict(devVector)
    # get the f1 score per category 
    f1 = metrics.f1_score(devLabel, devPred, average = None)
    # print the classification report
    print("Classification report:\n")
    devPred = modelLogit.predict(devVector)
    print(classification_report(devLabel, devPred))

    # get the prediction probabilities
    devPredP = modelLogit.predict_proba(devVector)

    print ("% Bad Predictions =", sum(devPred != devLabel)/len(devLabel), '\n')
    logitScore = round(modelLogit.score(devVector, devLabel), 4)
    print("For C = ", c, "Logistic regression accuracy:", logitScore)
    


Classification report:

             precision    recall  f1-score   support

      False       0.75      0.98      0.85       740
       True       0.45      0.06      0.10       260

avg / total       0.67      0.74      0.65      1000

% Bad Predictions = 0.263 

For C =  100 Logistic regression accuracy: 0.737


In [95]:
#TFIDF results: subredits only

reg = [100]
for c in reg:     
    RmodelLogit = LogisticRegression(penalty='l2', C=c )
    RmodelLogit.fit(RtrainVector, trainLabel)
    RdevPred = RmodelLogit.predict(RdevVector)
    # get the f1 score per category 
    f1 = metrics.f1_score(devLabel, RdevPred, average = None)
    # print the classification report
    print("Classification report:\n")
    RdevPred = RmodelLogit.predict(RdevVector)
    print(classification_report(devLabel, RdevPred))

    # get the prediction probabilities
    RdevPredP = RmodelLogit.predict_proba(RdevVector)

    print ("SubRedits % Bad Predictions =", sum(RdevPred != devLabel)/len(devLabel), '\n')
    logitScore = round(RmodelLogit.score(RdevVector, devLabel), 4)
    print("For C = ", c, "Logistic regression accuracy on SubRedits:", logitScore)



Classification report:

             precision    recall  f1-score   support

      False       0.74      0.86      0.80       740
       True       0.29      0.16      0.21       260

avg / total       0.63      0.68      0.64      1000

SubRedits % Bad Predictions = 0.322 

For C =  100 Logistic regression accuracy on SubRedits: 0.678


## Naive Bayes

In [8]:
# Multinomial NB

alpha = 0.01

clf = BernoulliNB(alpha=alpha)
clf.fit(tVector, trainLabel)
test_predicted_labels = clf.predict(dVector) 
print ('Bernoulli NB using raw data with alpha = %1.3f:' %alpha, metrics.accuracy_score(devLabel,test_predicted_labels) )

clf.fit(tVector_p, trainLabel)
test_predicted_labels = clf.predict(dVector_p) 
print ('Bernoulli NB using processed data  with alpha = %1.3f:' %alpha, metrics.accuracy_score(devLabel,test_predicted_labels) )


Bernoulli NB using raw data with alpha = 0.010: 0.719
Bernoulli NB using processed data  with alpha = 0.010: 0.708


## Logistic Regression More Feature Selection

In [21]:
# get the best regularization
regStrength = [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 6.0, 10.0]


for c in regStrength:
    modelLogit = LogisticRegression(penalty='l1', C=c)
    modelLogit.fit(tVector_p, trainLabel)
    logitScore = round(modelLogit.score(dVector_p, devLabel), 4)
    print("For C = ", c, "Logistic regression accuracy:", logitScore)

# although the best score comes from c=.001, the bet F1-score 
# comes from c=.5, and this gives better weight options
modelLogit = LogisticRegression(penalty='l1', C=.001, tol = .1)
modelLogit.fit(tVector_p, trainLabel)

print(max(modelLogit.coef_[0]))
numWeights = 5

sortIndex = np.argsort(modelLogit.coef_)
iLen = len(sortIndex[0])
print("\nTop", numWeights, "Weighted Features:")

for index in range((iLen - numWeights) , iLen):
    lookup = sortIndex[0][index]
    print(lookup)
    weight =  modelLogit.coef_[0][lookup]
    print(vectorizer.get_feature_names()[sortIndex[0][index]], weight)
    
print("Majority Class Probability:", 1 - sum(devLabel)/len(devLabel))
print(devLabel[:20])
print(modelLogit.predict(dVector_p)[:20])

For C =  0.0001 Logistic regression accuracy: 0.74
For C =  0.001 Logistic regression accuracy: 0.74
For C =  0.01 Logistic regression accuracy: 0.74
For C =  0.1 Logistic regression accuracy: 0.734
For C =  0.5 Logistic regression accuracy: 0.673
For C =  1.0 Logistic regression accuracy: 0.652
For C =  2.0 Logistic regression accuracy: 0.653
For C =  6.0 Logistic regression accuracy: 0.644
For C =  10.0 Logistic regression accuracy: 0.646
0.0

Top 5 Weighted Features:
3500
gamet 0.0
3501
gamin 0.0
3502
gamut 0.0
3484
galwa 0.0
10490
zzm 0.0
Majority Class Probability: 0.74
0     False
1     False
2     False
3     False
4     False
5      True
6     False
7     False
8     False
9      True
10     True
11    False
12    False
13    False
14    False
15    False
16     True
17    False
18     True
19    False
Name: requester_received_pizza, dtype: bool
[False False False False False False False False False False False False
 False False False False False False False False]


# Future Steps

* More data pre-processing (looking for newer features too)
* Explore PCA/LSA
* Ideas on features
    - Combination of words
    - Pruning
    - Timing (of requests)
    - Location
    